In [2]:
import moritz.lindner.masterarbeit.epa.construction.builder.ExtendedPrefixAutomatonBuilder
import moritz.lindner.masterarbeit.epa.construction.builder.SampleEventMapper
import moritz.lindner.masterarbeit.epa.*
import moritz.lindner.masterarbeit.epa.domain.*
import java.io.File

val epa = ExtendedPrefixAutomatonBuilder<Long>()
        .setFile(File("/Users/moritzlindner/programming/Masterarbeit/epa-visualizer/epa/src/test/resources/simple.xes"))
        .setEventLogMapper(SampleEventMapper())
        .build()

epa

simple.xes
j,i,root,a,f,d,e,h,c,b,g
a,b,c,d,e,f,g,h,i,j
Transition(start=d, activity=g, end=g),Transition(start=e, activity=h, end=h),Transition(start=b, activity=d, end=d),Transition(start=c, activity=e, end=e),Transition(start=c, activity=f, end=f),Transition(start=root, activity=a, end=a),Transition(start=a, activity=b, end=b),Transition(start=a, activity=c, end=c),Transition(start=h, activity=i, end=i),Transition(start=h, activity=j, end=j)
j:4,i:2,root:0,a:1,f:3,d:1,e:2,h:2,c:2,b:1,g:1
j:Event(activity=j, timestamp=1745625840000, caseIdentifier=3, predecessor=Event(activity=h, timestamp=1745625780000, caseIdentifier=3, predecessor=null)),i:Event(activity=i, timestamp=1745625840000, caseIdentifier=2, predecessor=Event(activity=h, timestamp=1745625780000, caseIdentifier=2, predecessor=null)),a:Event(activity=a, timestamp=1745625600000, caseIdentifier=1, predecessor=null),Event(activity=a, timestamp=1745625600000, caseIdentifier=2, predecessor=null),Event(activity=a, timestamp=174562

In [57]:
data class StateTransition(
    val parent: State,
    val state: State,
    val children: List<State>,
    val single: Boolean,
)

data class SyntheticState(
    val chain: List<State>,
    val newState: State,
    val children: List<State>,
    val parent: State,
    val seq: Set<Event<Long>>,
    val par: Int,
)

data class ChainInformation(
    val chain: List<State>,
    var parent: State,
    var children: List<State>
) {
    fun toSynthetic(extendedPrefixAutomaton: ExtendedPrefixAutomaton<Long>): SyntheticState {
        return SyntheticState(
            chain = chain,
            parent = parent,
            newState = chain.reduce { a, b ->
                State.PrefixState(
                    from = (a as State.PrefixState).from,
                    via = chain.fold(Activity("")) { a, b ->
                        Activity(a.name + (b as State.PrefixState).name)
                    }
                )
            },
            children = children,
            seq = chain.fold(emptySet()) { a, b -> a + epa.sequence(b) },
            par = epa.partition(chain.first()),
        )
    }
}

In [39]:
val childrenByParent = epa.transitions.groupBy { it.start }.mapValues { it.value.map { it.end } }
childrenByParent

{d=[g], e=[h], b=[d], c=[e, f], root=[a], a=[b, c], h=[i, j]}

In [40]:
val parentByChild = epa.transitions.groupBy { it.end }.mapValues { it.value.map { it.start }.first() }
parentByChild

{g=d, h=e, d=b, e=c, f=c, a=root, b=a, c=a, i=h, j=h}

In [41]:
val childrenByParentWithOneChildren = childrenByParent.filterValues { it.size == 1 }.filterKeys { it as? State.PrefixState != null }.mapValues { it.value.first() as State.PrefixState }
childrenByParentWithOneChildren

{d=g, e=h, b=d}

In [58]:
val stateTransitions = epa.states.mapNotNull { state ->
    val children = childrenByParent[state] ?: emptyList()
    StateTransition(
        parent = parentByChild[state] ?: return@mapNotNull null,
        state = state,
        children = children,
        single = children.size == 1
    )
}
stateTransitions

[StateTransition(parent=h, state=j, children=[], single=false), StateTransition(parent=h, state=i, children=[], single=false), StateTransition(parent=root, state=a, children=[b, c], single=false), StateTransition(parent=c, state=f, children=[], single=false), StateTransition(parent=b, state=d, children=[g], single=true), StateTransition(parent=c, state=e, children=[h], single=true), StateTransition(parent=e, state=h, children=[i, j], single=false), StateTransition(parent=a, state=c, children=[e, f], single=false), StateTransition(parent=a, state=b, children=[d], single=true), StateTransition(parent=d, state=g, children=[], single=false)]

In [59]:
fun followChain(a: State, mappings: Map<State, List<State>>, acc: List<State>): List<State> {
    if (mappings[a] != null) {
        val n = mappings[a]!!.first()
        return if (mappings[a]!!.size == 1) followChain(n, mappings, acc + n)
        else acc
    } else return acc
}

fun filterLongestChains(chains: List<ChainInformation>): List<ChainInformation> {
    return chains.filter { candidate ->
        chains.none { other ->
            other != candidate && isSubchain(candidate.chain, other.chain)
        }
    }
}

private fun isSubchain(subChain: List<State>, mainChain: List<State>): Boolean {
    if (subChain.size >= mainChain.size) return false

    // Check if subChain appears as a contiguous subsequence in mainChain
    for (i in 0..mainChain.size - subChain.size) {
        if (mainChain.subList(i, i + subChain.size) == subChain) {
            return true
        }
    }
    return false
}

In [60]:
fun buildChains(): List<ChainInformation> {
    return filterLongestChains(
        stateTransitions
        .filter { it.single }
        .map { stateTransition ->
            ChainInformation(
                chain = listOf(stateTransition.state) + followChain(
                    stateTransition.state,
                    childrenByParent,
                    emptyList()
                ),
                parent = stateTransition.parent,
                children = stateTransition.children,
            )
        }
    )
}

val chainInformation = buildChains()
chainInformation

[ChainInformation(chain=[e, h], parent=c, children=[h]), ChainInformation(chain=[b, d, g], parent=a, children=[d])]

In [64]:
val synthetics = chainInformation.map { it.toSynthetic(epa) }
synthetics

[SyntheticState(chain=[e, h], newState=eh, children=[h], parent=c, seq=[Event(activity=e, timestamp=1745625720000, caseIdentifier=2, predecessor=Event(activity=c, timestamp=1745625660000, caseIdentifier=2, predecessor=null)), Event(activity=e, timestamp=1745625720000, caseIdentifier=3, predecessor=Event(activity=c, timestamp=1745625660000, caseIdentifier=3, predecessor=null)), Event(activity=h, timestamp=1745625780000, caseIdentifier=2, predecessor=Event(activity=e, timestamp=1745625720000, caseIdentifier=2, predecessor=null)), Event(activity=h, timestamp=1745625780000, caseIdentifier=3, predecessor=Event(activity=e, timestamp=1745625720000, caseIdentifier=3, predecessor=null))], par=2), SyntheticState(chain=[b, d, g], newState=bdg, children=[d], parent=a, seq=[Event(activity=b, timestamp=1745625660000, caseIdentifier=1, predecessor=Event(activity=a, timestamp=1745625600000, caseIdentifier=1, predecessor=null)), Event(activity=d, timestamp=1745625720000, caseIdentifier=1, predecessor=E

In [98]:
synthetics.ma

{g=d, h=e, d=b, e=c, f=c, a=root, b=a, c=a, i=h, j=h}

In [66]:
val oldToNewStateMap = mutableMapOf<State, State>()
synthetics.forEach { synthetic ->
    synthetic.chain.forEach { oldState ->
        oldToNewStateMap[oldState] = synthetic.newState
    }
}
oldToNewStateMap

{e=eh, h=eh, b=bdg, d=bdg, g=bdg}

In [77]:
val statesInChains = synthetics.flatMap { it.chain }.toSet()
statesInChains

[e, h, b, d, g]

In [97]:
// Build new states set
val newStates = mutableSetOf<State>()

// Add states not in chains
epa.states.filter { it !in statesInChains }.forEach { state ->
    newStates.add(state)
}

newStates

[j, i, root, a, f, c]

In [95]:

// Add synthetic states
synthetics.forEach { synthetic ->
    newStates.add(synthetic.newState)
}

newStates

[j, i, root, a, f, c, eh, bdg]

In [96]:
// Update children's .from references
// Create set of states that will become synthetic
val statesThatBecomeSynthetic = synthetics.map { it.chain }.flatten().toSet()

synthetics.forEach { synthetic ->
    synthetic.children.forEach { child ->
        if (child is State.PrefixState && child !in statesThatBecomeSynthetic) {
            val updatedChild = State.PrefixState(
                from = synthetic.newState,
                via = child.via
            )
            newStates.add(updatedChild)
            oldToNewStateMap[child] = updatedChild
        }
    }
}
newStates.map { it to (it as? State.PrefixState)?.from }

[(j, h), (i, h), (root, null), (a, root), (f, c), (c, a), (eh, c), (bdg, a)]

In [190]:
val preActivities = epa.activities
preActivities

[a, b, c, d, e, f, g, h, i, j]

In [191]:
val preSeqByState = preStates.associateWith { epa.sequence(it) }
preSeqByState

{j=[Event(activity=j, timestamp=1745625840000, caseIdentifier=3, predecessor=Event(activity=h, timestamp=1745625780000, caseIdentifier=3, predecessor=null))], i=[Event(activity=i, timestamp=1745625840000, caseIdentifier=2, predecessor=Event(activity=h, timestamp=1745625780000, caseIdentifier=2, predecessor=null))], root=[], a=[Event(activity=a, timestamp=1745625600000, caseIdentifier=1, predecessor=null), Event(activity=a, timestamp=1745625600000, caseIdentifier=2, predecessor=null), Event(activity=a, timestamp=1745625600000, caseIdentifier=3, predecessor=null), Event(activity=a, timestamp=1745625600000, caseIdentifier=4, predecessor=null)], f=[Event(activity=f, timestamp=1745625720000, caseIdentifier=4, predecessor=Event(activity=c, timestamp=1745625660000, caseIdentifier=4, predecessor=null))], h=[Event(activity=h, timestamp=1745625780000, caseIdentifier=2, predecessor=Event(activity=e, timestamp=1745625720000, caseIdentifier=2, predecessor=null)), Event(activity=h, timestamp=1745625

[[e, h], [b, d, g]]

In [193]:
val activityByChain = chains.associateWith { chain ->
    chain.fold(Activity("")) { a, b ->
        Activity(a.name + (b as State.PrefixState).name)
    }
}
activityByChain

{[e, h]=eh, [b, d, g]=bdg}

In [194]:
val newActivities = preActivities + activityByChain.values
newActivities

[a, b, c, d, e, f, g, h, i, j, eh, bdg]

In [195]:
val syntheticStateByChain = chains.associateWith { chain ->
    chain.reduce { a, b ->
        State.PrefixState(
            from = (a as State.PrefixState).from,
            via = activityByChain[chain]!!
        )
    }
}
syntheticStateByChain

{[e, h]=eh, [b, d, g]=bdg}

In [196]:
val synthetics = syntheticStateByChain.values
synthetics

[eh, bdg]

In [206]:
val childrenBySynthetic = chains.associateWith { chain ->
    val last = chain.last()
    val targets = (childrenByParent[last] ?: emptyList())

    targets.map { target ->
        // if target is synthetic
        if (chains.flatten().contains(target)) {
            println("yes")
            val newFrom = syntheticByChainparts[target]!!
            println("newFrom for $target is $newFrom")
            newFrom
        } else State.PrefixState(
            from = syntheticStateByChain[chain]!!,
            via = (target as State.PrefixState).via
        )
    }
}.mapKeys { syntheticStateByChain[it.key]!! }

[[eh, eh], []]

In [127]:
import moritz.lindner.masterarbeit.epa.domain.Event

val seqByChain = chains.associateWith { chain ->
    chain.fold(emptyList<Event<Long>>()) { acc, state ->
        acc + epa.sequence(state)
    }
}
seqByChain.mapKeys { syntheticStateByChain[it.key] }

{fe=[Event(activity=f, timestamp=1745629440000, caseIdentifier=2, predecessor=Event(activity=d, timestamp=1745629380000, caseIdentifier=2, predecessor=null)), Event(activity=e, timestamp=1745629500000, caseIdentifier=2, predecessor=Event(activity=f, timestamp=1745629440000, caseIdentifier=2, predecessor=null))], bcd=[Event(activity=b, timestamp=1745625660000, caseIdentifier=1, predecessor=Event(activity=a, timestamp=1745625600000, caseIdentifier=1, predecessor=null)), Event(activity=b, timestamp=1745636460000, caseIdentifier=4, predecessor=Event(activity=a, timestamp=1745636400000, caseIdentifier=4, predecessor=null)), Event(activity=c, timestamp=1745625720000, caseIdentifier=1, predecessor=Event(activity=b, timestamp=1745625660000, caseIdentifier=1, predecessor=null)), Event(activity=c, timestamp=1745636520000, caseIdentifier=4, predecessor=Event(activity=b, timestamp=1745636460000, caseIdentifier=4, predecessor=null)), Event(activity=d, timestamp=1745625780000, caseIdentifier=1, pred

In [121]:
val parByChain = chains.associateWith { chain ->
    epa.partition(chain.first())
}
parByChain

{[f, e]=2, [b, c, d]=1, [c, b, d]=2, [f, e]=1}

In [129]:
val syntheticStateByChainStart = syntheticStateByChain.mapKeys { a -> a.key.first() }
syntheticStateByChainStart

{f=fe, b=bcd, c=cbd, f=fe}

In [170]:
val syntheticByParent = buildMap<State, List<State>> {
    childrenByParent.forEach { (parent, children) ->
        children.forEach { child ->
            if (syntheticStateByChainStart[child] != null) {
                val alreadyPresnt = get(parent) ?: emptyList<State>()
                put(parent, listOf(syntheticStateByChainStart[child]!!) + alreadyPresnt)
            }
        }
    }
}
syntheticByParent

{a=[bcd]}

In [ ]:
fun <T> mergeMaps(
    first: Map<T, List<T>>,
    second: Map<T, List<T>>
): Map<T, List<T>> {
    return (first.keys + second.keys).associateWith { key ->
        (first[key].orEmpty() + second[key].orEmpty())
    }
}

In [175]:
val mappingWithouthSynthStyle = buildMap<State, List<State>> {
    childrenByParent.forEach { parent, children ->
        if (childrenByParentWithOneChildren[parent] != null) return@forEach
        if (chains.flatten().contains(parent)) return@forEach
        val newChildren = children.filter {
            childrenByParentWithOneChildren[it] == null
        }
        put(parent, newChildren)
    }

    syntheticByParent.forEach { parent, synthetic ->
        val alreadyPreeasent = get(parent) ?: emptyList()

        put(parent, alreadyPreeasent + synthetic)
    }

    putAll(childrenBySynthetic)
}

mappingWithouthSynthStyle.values.flatten()

[e, bcd]

In [61]:
val transitions = mappingWithouthSynthStyle.flatMap { (parent, children) ->
    children.map { child ->
        Transition(
            start = parent,
            activity = (child as State.PrefixState).via,
            end = child
        )
    }
}

[Transition(start=c, activity=f, end=f), Transition(start=c, activity=eh, end=eh), Transition(start=root, activity=a, end=a), Transition(start=a, activity=c, end=c), Transition(start=a, activity=bdg, end=bdg), Transition(start=eh, activity=i, end=i), Transition(start=eh, activity=j, end=j)]

In [172]:
val syntheticByChainparts = buildMap {
    syntheticStateByChain.forEach { (chain, synth) ->
        chain.forEach { state ->
            put(state, synth)
        }
    }
}
syntheticByChainparts

{a=abc, b=abc, c=abc, d=de, e=de, b=bcde, c=bcde, d=bcde, e=bcde}

In [83]:
preStates
    .map { state ->
        if (state is State.PrefixState) {
            val from = state.from
            if (chains.flatten().contains(from)) {
                val newFrom = syntheticForPart[from]!!
                State.PrefixState(
                    from = newFrom,
                    via = state.via
                )
            } else state
        } else {
            state
        }
    }.map { (it as? State.PrefixState)?.from }

[eh, eh, null, root, c, eh, a, bdg]

In [ ]:
val prePartitionByState = epa.states.associateWith { epa.partition(it) }
prePartitionByState